In [8]:
import numpy as np
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter

class UnionFind:
    def __init__(self, n):
        """Inicializa estrutura de conjuntos disjuntos."""
        self.parent = list(range(n))

    def find(self, u):
        """Encontra o representante do conjunto de u com path compression."""
        if self.parent[u] != u:
            self.parent[u] = self.find(self.parent[u])
        return self.parent[u]

    def union(self, u, v):
        """Une os conjuntos de u e v."""
        ru, rv = self.find(u), self.find(v)
        if ru != rv:
            self.parent[rv] = ru
            return True
        return False

def generate_boruvka_frames(points, verbose=True):
    """Executa o algoritmo de Borůvka e gera os frames para animação."""
    n = len(points)
    dist_matrix = squareform(pdist(points))
    uf = UnionFind(n)
    mst_edges = []
    frames = []
    num_components = n

    while num_components > 1:
        cheapest = [None] * n
        for u in range(n):
            for v in range(n):
                if u != v and uf.find(u) != uf.find(v):
                    root = uf.find(u)
                    if cheapest[root] is None or dist_matrix[u][v] < dist_matrix[cheapest[root][0]][cheapest[root][1]]:
                        cheapest[root] = (u, v)

        new_edges = []
        for edge in cheapest:
            if edge is not None:
                u, v = edge
                if uf.find(u) != uf.find(v):
                    new_edges.append((u, v))

        frames.append((np.array(points), mst_edges.copy(), new_edges))

        for u, v in new_edges:
            if uf.union(u, v):
                mst_edges.append((u, v))
                num_components -= 1

    frames.append((np.array(points), mst_edges.copy(), []))

    if verbose:
        print("Arestas da Árvore Geradora Mínima (MST):")
        for u, v in mst_edges:
            print(f"{u} -- {v} (distância: {dist_matrix[u][v]:.4f})")

    return frames

def animate_boruvka(frames, filename="boruvka_mst.mp4"):
    """Cria e salva a animação da construção da MST."""
    fig, ax = plt.subplots()

    def update(frame):
        points, mst_edges, new_edges = frame
        ax.clear()
        ax.set_title(f"Etapa da MST (arestas: {len(mst_edges)})")
        ax.scatter(points[:, 0], points[:, 1], c='black')
        for u, v in mst_edges:
            ax.plot([points[u][0], points[v][0]], [points[u][1], points[v][1]], 'b-', linewidth=2)
        for u, v in new_edges:
            ax.plot([points[u][0], points[v][0]], [points[u][1], points[v][1]], 'r--', linewidth=1.5)
        ax.axis('equal')
        ax.grid(True)

    anim = FuncAnimation(fig, update, frames=frames, interval=1500)
    writer = FFMpegWriter(fps=1)
    anim.save(filename, writer=writer)
    plt.close(fig)


# Para testar o algoritmo, mude a váriavel "pts" abaixo e rode o algoritmo. 
Se quiser uma simulação visual com o matplotlib, mude a variável "animated" para "True". Se quiser gerar pontos aleatórios, utilize a função "generate_random_points" como pode ser visto abaixo. Caso contrário, passe uma array de pontos no formato [(a,b), (c,d), (e,f), ... (y,z)]

In [9]:
random.seed(42)
def generate_random_points(num_points):
    return [(random.uniform(-10, 10), random.uniform(-10, 10)) for _ in range(num_points)]

In [10]:
# MUDE ESSAS VARIÁVEIS
points = generate_random_points(50)
animated = True

# Exemplo de uso
if __name__ == "__main__":
    np.random.seed(0)
    frames = generate_boruvka_frames(points, verbose=True)

    if animated:
        animate_boruvka(frames, "boruvka_mst.mp4")

Arestas da Árvore Geradora Mínima (MST):
0 -- 20 (distância: 1.3678)
1 -- 39 (distância: 0.5104)
2 -- 10 (distância: 1.4517)
3 -- 9 (distância: 2.3081)
4 -- 11 (distância: 2.9977)
5 -- 8 (distância: 1.6786)
6 -- 22 (distância: 1.2632)
7 -- 27 (distância: 1.2843)
9 -- 20 (distância: 2.2398)
10 -- 15 (distância: 0.6324)
11 -- 25 (distância: 1.2353)
12 -- 36 (distância: 1.6753)
13 -- 44 (distância: 0.9927)
14 -- 18 (distância: 0.4674)
16 -- 41 (distância: 1.7897)
17 -- 8 (distância: 3.2483)
19 -- 14 (distância: 0.5992)
21 -- 39 (distância: 1.1307)
22 -- 23 (distância: 0.9983)
24 -- 37 (distância: 0.7318)
25 -- 42 (distância: 0.6165)
26 -- 35 (distância: 1.1269)
28 -- 45 (distância: 2.3765)
29 -- 47 (distância: 1.9980)
30 -- 48 (distância: 2.2216)
31 -- 27 (distância: 2.3660)
32 -- 15 (distância: 1.1691)
33 -- 44 (distância: 2.7790)
34 -- 39 (distância: 1.0961)
38 -- 43 (distância: 1.9448)
40 -- 24 (distância: 2.5255)
43 -- 48 (distância: 0.3928)
45 -- 8 (distância: 2.3437)
46 -- 38 (distâ